In [1]:
import pandas as pd

from get_data import CECADScraper
#from clear_data import clear_br, clear_uf
from Filters.urls_dict import urls
from Filters.filters_dict import *
from Utils.calculators import convert_date_pt

In [2]:
# Adquirindo dados através do CECAD
uf_fam = dict()
uf_pessoas = dict() 
scraper = CECADScraper('Utils/chromedriver', urls['CECAD'], headless=True)
for estado in estados_brasil:
    filter = {
        'geo': estado,
        'var1': 'Bloco 1 - Recebe PAB',
        'var2': 'Bloco 1 - Faixa da renda familiar per capita'
    }
    fam, pessoas = scraper.scrape_table(filter)
    territorio_referencia = scraper.get_territorio_referencia()
    uf_fam[estado]=fam
    uf_pessoas[estado]=pessoas
    print(f'Requerido: {estado}; Concluído: {territorio_referencia}')

# Adquirindo e convertendo a data de referência para formato datetime    
DATA_REFERENCIA = convert_date_pt(scraper.get_data_referencia())

filter = {
    'geo': 'Brasil',
    'var1': 'Bloco 1 - Recebe PAB',
    'var2': 'Bloco 1 - Faixa da renda familiar per capita'
}
html_br_fam, html_br_pes = scraper.scrape_table(filter)
scraper.quit()

Requerido: AC; Concluído: AC-Acre
Requerido: AL; Concluído: AL-Alagoas
Requerido: AP; Concluído: AP-Amapá
Requerido: AM; Concluído: AM-Amazonas
Requerido: BA; Concluído: BA-Bahia
Requerido: CE; Concluído: CE-Ceará
Requerido: DF; Concluído: DF-DistritoFederal
Requerido: ES; Concluído: ES-EspíritoSanto
Requerido: GO; Concluído: GO-Goiás
Requerido: MA; Concluído: MA-Maranhão
Requerido: MT; Concluído: MT-MatoGrosso
Requerido: MS; Concluído: MS-MatoGrossodoSul
Requerido: MG; Concluído: MG-MinasGerais
Requerido: PA; Concluído: PA-Pará
Requerido: PB; Concluído: PB-Paraíba
Requerido: PR; Concluído: PR-Paraná
Requerido: PE; Concluído: PE-Pernambuco
Requerido: PI; Concluído: PI-Piauí
Requerido: RJ; Concluído: RJ-RiodeJaneiro
Requerido: RN; Concluído: RN-RioGrandedoNorte
Requerido: RS; Concluído: RS-RioGrandedoSul
Requerido: RO; Concluído: RO-Rondônia
Requerido: RR -; Concluído: RR-Roraima
Requerido: SC; Concluído: SC-SantaCatarina
Requerido: SP; Concluído: SP-SãoPaulo
Requerido: SE; Concluído: S

### Unindo dados dos estados em um único dataframe

In [8]:
clear = pd.read_html(uf_fam['ES'], header=1, decimal=',', thousands='.')[0]

In [9]:
clear

,Faixa da renda familiar per capita,Não,Sim,Sem Resposta,TOTAL
0,Extrema Pobreza,38010.0,205302.0,0.0,243312.0
1,Pobreza,27350.0,48970.0,0.0,76320.0
2,Baixa Renda,0.0,0.0,0.0,0.0
3,Acima de 1/2 S.M.,0.0,0.0,0.0,0.0
4,Sem Resposta,0.0,0.0,0.0,0.0
5,TOTAL,65360.0,254272.0,0.0,319632.0
6,NaN,NaN,NaN,NaN,NaN


In [10]:
clear = clear[clear["Faixa da renda familiar per capita"].isin(["Extrema Pobreza", "Pobreza"])]
clear['c'] = 'BR'

In [11]:
clear=clear.pivot(index='c', columns='Faixa da renda familiar per capita').reset_index()
clear.columns = ['_'.join(col).rstrip('_') for col in clear.columns.values]
#clear['ano'] = DATA_REFERENCIA.year
#clear['mes'] = DATA_REFERENCIA.month

In [14]:
clear.drop(columns=['c'], inplace=True)

In [15]:
clear

,Não_Extrema Pobreza,Não_Pobreza,Sim_Extrema Pobreza,Sim_Pobreza,Sem Resposta_Extrema Pobreza,Sem Resposta_Pobreza,TOTAL_Extrema Pobreza,TOTAL_Pobreza
0,2.564.804,1.403.379,15.879.958,1.796.866,0.0,0.0,18.444.762,3.200.245


In [33]:
clear['UF'] = 'AC'
clear = clear[clear["Faixa da renda familiar per capita"].isin(["Extrema Pobreza", "Pobreza"])]
clear=clear.pivot(index='UF', columns='Faixa da renda familiar per capita').reset_index()
clear.columns = ['_'.join(col).rstrip('_') for col in clear.columns.values]
clear['ano'] = DATA_REFERENCIA.year
clear['mes'] = DATA_REFERENCIA.month

In [39]:
clear

,UF,Não_Extrema Pobreza,Não_Pobreza,Sim_Extrema Pobreza,Sim_Pobreza,Sem Resposta_Extrema Pobreza,Sem Resposta_Pobreza,TOTAL_Extrema Pobreza,TOTAL_Pobreza,ano,mes
0,AC,14.271,5.846,100.698,8.162,0.0,0.0,114.969,14.008,2022,5


In [22]:
dfs = []
for data in uf_fam.values():
    uf = pd.read_html(data, header=1)[0]
    uf.dropna(inplace=True)
    uf.rename(columns={'Unnamed: 0':'UF'}, inplace=True)
    uf = uf[uf['UF']!='TOTAL']
    dfs.append(uf)
familias_uf = pd.concat(dfs)

In [25]:
familias_uf.reset_index(drop=True, inplace=True)

In [26]:
familias_uf

,UF,Não,Sim,Sem Resposta,TOTAL
0,AC-Acre,41.564,376.376,0.0,417.94
1,AL-Alagoas,119.227,1.305.110,0.0,1.424.337
2,AP-Amapá,56.498,303.133,0.0,359.631
3,AM-Amazonas,185.317,1.632.821,0.0,1.818.138
4,BA-Bahia,652.937,5.785.555,0.0,6.438.492
5,CE-Ceará,341.895,3.514.925,0.0,3.856.820
6,DF-DistritoFederal,48.237,355.217,0.0,403.454
7,ES-EspíritoSanto,135.021,719.042,0.0,854.063
8,GO-Goiás,242.849,1.148.307,0.0,1.391.156
9,MA-Maranhão,299.258,3.259.193,0.0,3.558.451


In [12]:
pd.concat(uf_fam.values())

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [5]:
import pandas as pd

In [35]:
html_br_fam

('Recebe PAB família TOTAL\nNão Sim Sem Resposta\nBrasil 3.968.183 17.676.824 0 21.645.007\nTOTAL 3.968.183 17.676.824 0 21.645.007',
 '<table class="table table-bordered table-striped">\n    <tbody><tr>\n    <th rowspan="2" style="vertical-align: middle;"></th>\n    <th colspan="3" style="text-align: center;">Recebe PAB família</th>\n    <th rowspan="2" class="text-center" style="vertical-align: middle;">TOTAL</th>\n    </tr><tr><th> Não</th><th> Sim</th><th>Sem Resposta</th></tr><tr><th>Brasil</th><td>3.968.183</td><td>17.676.824</td><td>0</td><td>21.645.007</td></tr><tr><th><b>TOTAL</b></th><td>3.968.183</td><td>17.676.824</td><td>0</td><td>21.645.007</td></tr><tr></tr></tbody></table>')

In [34]:
table = pd.read_html(html_br_fam, header=1)
table.rename(columns={'Unnamed: 0':'Territorio'}, inplace=True)

TypeError: cannot parse from 'tuple'

In [21]:
table.dropna()

,Territorio,Não,Sim,Sem Resposta,TOTAL
0,Brasil,3.968.183,17.676.824,0.0,21.645.007
1,TOTAL,3.968.183,17.676.824,0.0,21.645.007
